## Created By : Shreya Dubey
## Data Preprocessing for Abtractive model

In [67]:
from attention import AttentionLayer
from nltk.corpus import stopwords
import nltk

In [5]:
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
import csv

# Loading data

def load_data(filename):
    with open(filename) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        headlines = []
        summaries = []
        articles = []
        for row in readCSV:
            headline = row[0]
            summary = row[1]
            article = row[2]

            headlines.append(headline)
            summaries.append(summary)
            articles.append(article)

    headlines.remove(headlines[0])
    summaries.remove(summaries[0])
    articles.remove(articles[0])
    
    return headlines,articles,summaries

# loading data from news_summary_2.csv
headlines, articles, summaries = load_data('news_summary_2.csv')


In [6]:
try:
    for i in range(len(articles)):
        if(articles[i]==''):
            if(summaries[i]!=''):
                articles[i] = summaries[i]
            else:
                articles.remove(articles[i])
                headlines.remove(headlines[i])
                summaries.remove(summaries[i])
except:
    print(i)
    
print("total = ",len(articles))

4515
total =  4515


In [7]:
# dictionary for expanding contractions

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [8]:
stop_words = set(stopwords.words('english')) 

def text_cleaner(text,num):
    newString = text.lower()
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])
    newString = newString.replace("\n"," ")
    newString = re.sub(r'[^a-zA-Z0-9_\s]+', '', newString)
    if(num==0):
        tokens = [w for w in newString.split() if not w in stop_words]
    else:
        tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

#def text_cleaner(text,num):
#    newString = text.lower()
#    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])  
#    newString = re.sub(r'[^a-zA-Z0-9_\s]+', '', newString)
#    if(num==0):
#        tokens = [w for w in newString.split() if not w in stop_words]
#    else:
#        tokens=newString.split()
#    long_words=[]
#    for i in tokens:
#        long_words.append(i)   
#    return (" ".join(long_words)).strip()

# cleaning the articles
cleaned_text = []
for t in articles:
    #cleaned_text.append(text_cleaner(t,0))
    cleaned_text.append(text_cleaner(t,1))

# cleaning the headlines
cleaned_headline = []
for t in headlines:
    cleaned_headline.append(text_cleaner(t,1))

In [9]:
print(stop_words)

{'ve', 'further', 'hadn', 'ma', 'ours', 'wasn', 'the', 'they', 'herself', 'was', 'that', 'me', "that'll", 'of', 'y', 'this', 'having', 'can', "wasn't", "needn't", 'above', 'as', 'in', 'our', 'which', 'to', 'because', 'does', 'him', 'won', 'what', 'over', 'against', "mightn't", "hasn't", 'were', 'own', 'ain', 'more', 'now', 'no', 'down', 'if', "you've", 'them', 'until', 'my', 'any', 'isn', 'hasn', 'didn', "didn't", 'couldn', 'so', 'by', 'you', 'there', 'below', 'up', 'should', 'about', 'and', 'her', 'once', 'do', 'at', 'o', 'few', 'mightn', 'same', 'is', 'we', 'aren', 'both', 'here', 'an', "isn't", 'who', 'not', 'where', 'mustn', 'yourself', 'yours', 'under', 'will', 'has', 'before', 'other', 're', 'themselves', "doesn't", 'while', 'whom', "you're", "couldn't", 'its', 'their', 'those', 'why', 'just', 'such', 'itself', 'after', 'very', "you'd", 'being', 'than', 'but', "weren't", 'needn', 'through', 'all', 'these', 'from', "hadn't", "mustn't", 'wouldn', 'too', 'be', 'each', 's', 'when', "

In [10]:
# Consolidating the processed data

data=pd.DataFrame()
data['cleaned_text']=cleaned_text
data['cleaned_headline']=cleaned_headline

In [11]:
data['cleaned_headline'][3614]
data['cleaned_text'][3614]

'sujal kahar 12yearold football enthusiast and goalkeeper from mumbai has been shortlisted to represent india at the global football for friendship f4f social programme it will be held in russias st petersburg in july india will join the f4f programme for the first time and will participate alongside over 60 other countries'

In [12]:
#Dropping empty rows

data.replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)

In [13]:
# analysis of the text and headline length so that we can fix a max lenth for both

import matplotlib.pyplot as plt

text_word_count = []
headline_word_count = []

# populate the lists with sentence lengths
for i in data['cleaned_text']:
      text_word_count.append(len(i))

for i in data['cleaned_headline']:
      headline_word_count.append(len(i))

length_df = pd.DataFrame({'text':text_word_count, 'headlines':headline_word_count})

length_df.hist(bins = 30)
plt.show()

<Figure size 640x480 with 2 Axes>

In [14]:
cnt=0
for i in data['cleaned_text']:
    if(len(i.split())<=600):
        cnt=cnt+1
print(cnt/len(data['cleaned_text']))

0.9231450719822812


In [15]:
# Fixing max length of text and headline
max_text_len=600
max_headline_len=12

In [16]:
cleaned_text =np.array(data['cleaned_text'])
cleaned_headline=np.array(data['cleaned_headline'])

short_text=[]
short_headline=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_headline[i].split())<=max_headline_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_headline.append(cleaned_headline[i])
        
df=pd.DataFrame({'text':short_text,'headline':short_headline})

In [17]:
# adding start of sentence token 'sost' and end of sentence token 'eost' to the headlines
df['headline'] = df['headline'].apply(lambda x : 'sost '+ x + ' eost')

In [18]:
export_csv = df.to_csv ('Processed2.csv', index = None, header=True)

In [42]:
# Split the dataset now into 90% training and 10% test

from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(np.array(df['text']),np.array(df['headline']),test_size=0.1,random_state=0,shuffle=True)

In [43]:
# Building tokenizer to convert word sequence into integer sequence

from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

# Analysing the number of rare words in the vocabulary (rule: any word that appears 3 or leass times is rare)

thresh=6

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

#prepare a tokenizer for articles on training data
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

#size of vocabulary ( +1 for padding token)
x_voc   =  x_tokenizer.num_words + 1

% of rare words in vocabulary: 80.9575402635432
Total Coverage of rare words: 12.387656718697322


In [44]:
# Building tokenizer for headline as well as we did for text

#prepare a tokenizer for headlines on training data
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

# Analysis for rare words in headlines (here threshold is 6 for rare words as the length of headlines is shorter.) 

thresh=6

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

#prepare a tokenizer for headlines on training data
y_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
y_tokenizer.fit_on_texts(list(y_tr))

#convert text sequences into integer sequences
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_headline_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_headline_len, padding='post')

#size of vocabulary
y_voc  =   y_tokenizer.num_words +1

% of rare words in vocabulary: 87.17829816832831
Total Coverage of rare words: 29.404908976480737


In [139]:
y_tokenizer.word_counts['sostok'],len(y_tr)

(3846, 3846)

In [140]:
# deleting the headlines that only have start and end tokens in them

ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)

ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

In [93]:
# Building model

from keras import backend as K 
K.clear_session()

latent_dim = 300
embedding_dim=100

# Encoder
encoder_inputs = Input(shape=(max_text_len,))

#embedding layer
enc_emb =  Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

# Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1200)         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1200, 100)    2161100     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 1200, 300),  481200      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer) 

In [94]:
abc = [encoder_outputs,decoder_outputs]
print(abc[0][2])
print(encoder_outputs[2])
print((abc[1][2], abc[0][2]))
print(decoder_outputs[2])

Tensor("strided_slice:0", shape=(1200, 300), dtype=float32)
Tensor("strided_slice_1:0", shape=(1200, 300), dtype=float32)
(<tf.Tensor 'strided_slice_2:0' shape=(?, 1147) dtype=float32>, <tf.Tensor 'strided_slice_3:0' shape=(1200, 300) dtype=float32>)
Tensor("strided_slice_4:0", shape=(?, 1147), dtype=float32)


In [95]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [96]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)

In [97]:

history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=5,callbacks=[es],batch_size=128, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

Train on 3946 samples, validate on 438 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/5


ResourceExhaustedError: OOM when allocating tensor with shape[1200,128,300] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu
	 [[{{node lstm_2/TensorArrayStack/TensorArrayGatherV3}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
